In [2]:
import os

In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI


e:\Programming\Langchain_oneshot\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.7, convert_system_message_to_human=True)

In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import memo

2

In [9]:
from langchain_core.tools import tool

In [11]:
@tool
def search(query:str):
    """call to the surf web"""
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "its 60 degrees and foggy"
    return "its 90 degress and sunny"

In [12]:
tools = [search]

In [13]:
tool_node =  ToolNode(tools)

In [14]:
def call_model(state:MessagesState):
    messages = state['messages']
    response =  model.invoke(messages)
    return {"messages":[response]}

In [17]:
from typing import Literal

In [18]:
def should_continue(state:MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

In [19]:
workflow = StateGraph(MessagesState)

In [20]:
workflow.add_node("agent", call_model)

In [21]:
workflow.add_node("tools",call_model)

In [24]:
workflow.add_conditional_edges(
    # First we define the start node. we use 'agent'
    # This means these are the edges taken after the 'agent' node called.
    "agents",
    #Next, we call in the function that will determine which node is called next
    should_continue,
)

ValueError: Branch with name `should_continue` already exists for node `agents`

In [26]:
workflow.add_edge('tools','agent')

In [27]:
checkpointer = MemorySaver()

In [28]:
app =  workflow.compile(checkpointer=checkpointer)

ValueError: Found edge starting at unknown node 'agents'

In [29]:
from IPython.display import display, Image
try:
    display(Image(app.get_graph().draw_mermaid()))
except Exception:
    pass

In [33]:
from langchain_core.messages import HumanMessage

In [34]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="what is the weather in sf")]},
     config={"configurable":{"thread_id":42}}
)

NameError: name 'app' is not defined